In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer,TFBertForSequenceClassification

f:\Sentiment\sent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing Datasets

In [2]:
amazon=pd.read_csv("../datasets/amazon.csv")
amazon2=pd.read_csv("../datasets/amazon2.csv")
uber=pd.read_csv("../datasets/uber.csv")
womens=pd.read_csv("../datasets/womens.csv")
air=pd.read_csv("../datasets/air_india.csv")

Data Preprocessing

In [3]:
amazon.head(3)

,index,product,helpful_count,total_comments,url,review_country,reviewed_at,review_text,review_rating,product_company,profile_name,review_title
0,0,Apple iPhone XR (64GB) - Black,"5,087 people found this helpful",24,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2018-12-12,NOTE:,3.0 out of 5 stars,Apple,Sameer Patil,"Which iPhone you should Purchase ? iPhone 8, X..."
1,1,Apple iPhone XR (64GB) - Black,"2,822 people found this helpful",6,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2018-11-17,Very bad experience with this iPhone xr phone....,1.0 out of 5 stars,Apple,Amazon Customer,Don't buy iPhone xr from Amazon.
2,2,Apple iPhone XR (64GB) - Black,"1,798 people found this helpful",0,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2019-01-27,Amazing phone with amazing camera coming from ...,5.0 out of 5 stars,Apple,A,Happy with the purchase


In [4]:
amazon['review_rating']=amazon['review_rating'].astype(str).str[0].astype(int)

In [5]:
amazon.head(3)

,index,product,helpful_count,total_comments,url,review_country,reviewed_at,review_text,review_rating,product_company,profile_name,review_title
0,0,Apple iPhone XR (64GB) - Black,"5,087 people found this helpful",24,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2018-12-12,NOTE:,3,Apple,Sameer Patil,"Which iPhone you should Purchase ? iPhone 8, X..."
1,1,Apple iPhone XR (64GB) - Black,"2,822 people found this helpful",6,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2018-11-17,Very bad experience with this iPhone xr phone....,1,Apple,Amazon Customer,Don't buy iPhone xr from Amazon.
2,2,Apple iPhone XR (64GB) - Black,"1,798 people found this helpful",0,https://www.amazon.in/Apple-iPhone-XR-64GB-Bla...,India,2019-01-27,Amazing phone with amazing camera coming from ...,5,Apple,A,Happy with the purchase


In [6]:
for i in amazon.columns:
    if i!="review_rating" and i!="review_text":
        amazon.drop([i],axis=1,inplace=True)
amazon.head()

,review_text,review_rating
0,NOTE:,3
1,Very bad experience with this iPhone xr phone....,1
2,Amazing phone with amazing camera coming from ...,5
3,So I got the iPhone XR just today. The product...,1
4,I've been an android user all my life until I ...,5


In [ ]:
amazon2.head()

In [7]:
for i in amazon2.columns:
    if i!="rating" and i!="verified_reviews":
        amazon2.drop([i],axis=1,inplace=True)
amazon2=amazon2.rename(columns={"rating":"review_rating","verified_reviews":"review_text"})
amazon2.head()

,review_rating,review_text
0,5,Love my Echo!
1,5,Loved it!
2,4,"Sometimes while playing a game, you can answer..."
3,5,I have had a lot of fun with this thing. My 4 ...
4,5,Music


In [ ]:
uber.head(3)

In [8]:
for i in uber.columns:
    if i!="content" and i!="score":
        uber.drop([i],axis=1,inplace=True)
uber=uber.rename(columns={"content":"review_text","score":"review_rating"})
uber.head()   

,review_text,review_rating
0,Good,5
1,Nice,5
2,Very convenient,5
3,Good,4
4,exllence,5


In [9]:
womens.head(3)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses


In [10]:
for i in womens.columns:
    if i!="Review Text" and i!="Rating":
        womens.drop([i],axis=1,inplace=True)
womens=womens.rename(columns={"Review Text":"review_text","Rating":"review_rating"})
womens.head()

,review_text,review_rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [11]:
air.head(3)

,published_date,rating,type,helpful_votes,title,text
0,2024-03-28T17:44:04-04:00,1,review,0,BAD SERVICE AND BAD HOSPITALITY,From their side my baggage is broken but still...
1,2024-03-28T17:24:01-04:00,1,review,0,AI 188 and AI 187 flight nightmare !,I took AI 188 noon -9th March 2024 - it was de...
2,2024-03-28T16:31:49-04:00,1,review,0,worst airline to travel dont waste ur money,one of worst flight to fly with book one way l...


In [12]:
for i in air.columns:
    if i!="text" and i!="rating":
        air.drop([i],axis=1,inplace=True)
air=air.rename(columns={"text":"review_text","rating":"review_rating"})
air.head(3)

,review_rating,review_text
0,1,From their side my baggage is broken but still...
1,1,I took AI 188 noon -9th March 2024 - it was de...
2,1,one of worst flight to fly with book one way l...


In [13]:
data=pd.concat([amazon,amazon2,uber,womens,air],axis=0)
data.head()

,review_text,review_rating
0,NOTE:,3
1,Very bad experience with this iPhone xr phone....,1
2,Amazing phone with amazing camera coming from ...,5
3,So I got the iPhone XR just today. The product...,1
4,I've been an android user all my life until I ...,5


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48486 entries, 0 to 4839
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_text    47637 non-null  object
 1   review_rating  48486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [15]:
data.isna().sum()

review_text      849
review_rating      0
dtype: int64

Filling missing Values

In [16]:
data['review_text'].fillna(data['review_text'].mode()[0],inplace=True)
data.isna().sum()

C:\Users\Anwar\AppData\Local\Temp\ipykernel_5384\652859361.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['review_text'].fillna(data['review_text'].mode()[0],inplace=True)


review_text      0
review_rating    0
dtype: int64

Suffeling dataset

In [17]:
data=data.sample(frac=1)
data.head()

,review_text,review_rating
4527,Really nice phone,5
32,She works well. Needs a learning command for ...,5
14960,I wore this tee under a comfy cardigan on a fl...,4
3716,Air India has a better luggage allowance than ...,3
1213,I love love it !,5


Reseting Index

In [18]:
data.reset_index(inplace=True)

In [19]:
data.drop(['index'],axis=1,inplace=True)
data.head()

,review_text,review_rating
0,Really nice phone,5
1,She works well. Needs a learning command for ...,5
2,I wore this tee under a comfy cardigan on a fl...,4
3,Air India has a better luggage allowance than ...,3
4,I love love it !,5


In [20]:
data.index

RangeIndex(start=0, stop=48486, step=1)

In [ ]:
def modify(review_rating):
    if review_rating>=4:
        return 1
    elif review_rating==3:
        return 2
    else:
        return 0
# positive 1,negative 0,neutral=2
data['review_rating']=data['review_rating'].apply(modify)
data.head()

,review_text,review_rating
0,Really nice phone,1
1,She works well. Needs a learning command for ...,1
2,I wore this tee under a comfy cardigan on a fl...,1
3,Air India has a better luggage allowance than ...,1
4,I love love it !,1


In [ ]:
data.to_csv("reviews.csv",index=False)

In [4]:
data=pd.read_csv("reviews.csv")

In [5]:
x=data['review_text'].tolist()
y=data['review_rating'].tolist()

In [6]:
# splitting the data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [7]:
# tokenization with bert
#  loading bert tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

f:\Sentiment\sent\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# tokenize the text
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=128, return_tensors='tf')

In [ ]:
# prepare tensorflow dataset
# convert y into tensors
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [10]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).shuffle(1000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

In [ ]:
# Load pretrained BERT model with 3 output labels (0 = neg, 1 = pos, 2 = neutral)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#   Early stopping inorder to prevent model overfitting
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2,restore_best_weights=True)

In [ ]:
model.fit(train_dataset,epochs=2,validation_data=test_dataset,callbacks=[early_stop])

In [ ]:
#  model Evaluation
model.evaluate(test_dataset)

In [ ]:
#  saving the finetuned model
model.save_pretrained("saved_bert_model")
tokenizer.save_pretrained("saved_bert_model")

In [ ]:
# loading the saved model
model=TFBertForSequenceClassification.from_pretrained("./myapp/saved_bert_model2")
tokenizer=BertTokenizer.from_pretrained("./myapp/saved_bert_model2")

Some layers from the model checkpoint at saved_bert_model1 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at saved_bert_model1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [14]:
# prediction
# new review
new=["its not good"]
# tokenize
inputs=tokenizer(new,truncation=True,padding=True,return_tensors="tf")
# predict
output=model.predict(dict(inputs)).logits
predicted_model=tf.argmax(output,axis=1).numpy()[0]
# map label for sentiment
label={0:"negative",1:"positive",2:"neutral"}
print("predicted sentiment is:",label[predicted_model])

1/1 [==============================] - 4s 4s/step
predicted sentiment is: negative


In [ ]:
data=pd.read_csv("reviews.csv")

x=data['review_text'].tolist()
y=data['review_rating'].tolist()

# splitting the data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# tokenization with bert
#  loading bert tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

# tokenize the text
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=128, return_tensors='tf')

# prepare tensorflow dataset
# convert y into tensors
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).shuffle(1000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

# Load pretrained BERT model with 3 output labels (0 = neg, 1 = pos, 2 = neutral)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

#   Early stopping inorder to prevent model overfitting
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2,restore_best_weights=True)

model.fit(train_dataset,epochs=2,validation_data=test_dataset,callbacks=[early_stop])

#  model Evaluation
model.evaluate(test_dataset)

#  saving the finetuned model
model.save_pretrained("saved_bert_model")
tokenizer.save_pretrained("saved_bert_model")

In [ ]:
# loading the saved model
model=TFBertForSequenceClassification.from_pretrained("./myapp/saved_bert_model2")
tokenizer=BertTokenizer.from_pretrained("./myapp/saved_bert_model2")

# prediction
# new review
new=["its not good"]
# tokenize
inputs=tokenizer(new,truncation=True,padding=True,return_tensors="tf")
# predict
output=model.predict(dict(inputs)).logits
predicted_model=tf.argmax(output,axis=1).numpy()[0]
# map label for sentiment
label={0:"negative",1:"positive",2:"neutral"}
print("predicted sentiment is:",label[predicted_model])